# Challenge 2: Model that predicts a gem's worth based on its characteristics

Author: Giacomo Rossi

Repository: [1giacomo/xtream-ai-assignment](https://github.com/1giacomo/xtream-ai-assignment)

LinkedIn: [giacomo-rossi-100](https://www.linkedin.com/in/giacomo-rossi-100/)

[Go to Challenge 1](https://colab.research.google.com/drive/1wkIHYESKVFetaFaZ9ZWnE3EZC6939grY)